In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import timedelta, date
import time
import openpyxl

In [13]:
df = pd.read_excel("../Text mining/merged_dataframe.xlsx")
na_df = df[df['lyrics'].isna()].reset_index()[['song_name', 'artist_name']]

In [14]:
na_df

,song_name,artist_name
0,Crank That (Soulja Boy),Soulja Boy Tell'em
1,Sweetest Girl (Dollar Bill),"Wyclef Jean Featuring Akon, Lil Wayne & Niia"
2,How Far We've Come,matchbox twenty
3,I Don't Wanna Be In Love (Dance Floor Anthem),Good Charlotte
4,Until The End Of Time,Justin Timberlake Duet With Beyonce
...,...,...
1395,Astronaut In The Ocean,Masked Wolf
1396,Heartbreak Anniversary,Giveon
1397,Like I Want You,Giveon
1398,Clouds,NF


In [15]:
wanted = na_df[1200:].reset_index()[['song_name', 'artist_name']]

In [17]:
# azlyric version

def scrape_lyrics(song_name, artist_name):
    
    base_url = 'https://www.azlyrics.com/lyrics/'
    
    artist_name=artist_name.split(" Featuring ")[0]
    artist_name=artist_name.split(" X ")[0]
    artist_name=artist_name.split(" x ")[0]
    artist_name=artist_name.split(" & ")[0]
    artist_name=artist_name.split(" + ")[0]
    artist_name=artist_name.replace("-", "")
    artist_name=artist_name.replace("!", "")
    artist_name=artist_name.replace(".", "")
    
    if artist_name == "Tyler, the Creator":
        artist_name = 'tyler the creator'
        
    artist_name=artist_name.split(", ")[0]
    
    if artist_name == "The Weekend":
        artist_name = 'weeknd'
        
    if artist_name == "The Kid Laroi":
        artist_name = 'kidlaroi'
        
    
    
    if artist_name == "YoungBoy Never Broke Again":
        artist_name = "NBA YoungBoy"
        
    if artist_name == "Beyonce":
        artist_name = "beyonceknowles"
    
    song_name = song_name.replace("'", "")
    song_name = song_name.replace("-", "")
    song_name = song_name.replace(".", "")
    song_name = song_name.replace("!", "")
    song_name = song_name.replace("&", "")
    song_name = song_name.replace("?", "")
    song_name = song_name.replace("(", "")
    song_name = song_name.replace(")", "") 
    
    artist_name = artist_name.lower().replace(" ", "")
    song_name = song_name.lower().replace(" ", "")
    
    url = f'{base_url}{artist_name}/{song_name}.html'

    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    lyrics_div = soup.find_all("div", attrs={"class": None, "id": None})
    lyrics = [lyric.getText() for lyric in lyrics_div]
    lyrics = " ".join(lyrics)
    time.sleep(5)
    
    return lyrics

In [18]:
all_lyric = []
for i in range(len(wanted)):
    try:
        if scrape_lyrics(wanted['song_name'][i], wanted['artist_name'][i]) == '': 
            print(wanted['artist_name'][i], wanted['song_name'][i], "-failed to find song-", i)
            all_lyric.append('X')
        else:    
            all_lyric.append(scrape_lyrics(wanted['song_name'][i], wanted['artist_name'][i])) 
    except:
        if scrape_lyrics(wanted['song_name'][i], wanted['artist_name'][i]) != '':
            all_lyric.append('')
        print(wanted['artist_name'][i], wanted['song_name'][i], "-failed to load-")

Dan + Shay I Should Probably Go To Bed -failed to find song- 10
Mulatto B*tch From da Souf -failed to find song- 17
Keith Urban Duet With P!nk One Too Many -failed to find song- 28
YFN Lucci Wet. (She Got That...) -failed to find song- 32
Zayn Better -failed to find song- 37
Chris Lane Big, Big Plans -failed to find song- 39
Karol G Ay, Dios Mio! -failed to find song- 40
21 Savage & Metro Boomin Featuring Young Thug Rich N*gga Sh*t -failed to find song- 46
Why Don't We Fallin' -failed to find song- 50
21 Savage & Metro Boomin Steppin On N*ggas -failed to find song- 56
Bebe Rexha Featuring Doja Cat Baby, I'm Jealous -failed to find song- 59
Lady A Champagne Night -failed to find song- 65
The Kid LAROI So Done -failed to find song- 67
Little Big Town Wine, Beer, Whiskey -failed to find song- 71
Ariana Grande Feat. Doja Cat & Megan Thee Stallion 34+35 -failed to find song- 73
DJ Chose Featuring BeatKing Thick -failed to find song- 88
The Kid LAROI Featuring YoungBoy Never Brok Again & Int

In [19]:
len(all_lyric)

200

In [20]:
wanted['lyrics'] = all_lyric

In [21]:
wanted.to_excel('billboard_lyrics.xlsx', index=False)